In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error as mse
import matplotlib
from matplotlib.collections import LineCollection
%matplotlib qt


data = np.genfromtxt(f'data/route_fup137_2.csv', delimiter=',', skip_header=1)


slc = 2
t = data[::slc, 0]
x = data[::slc, 1]
y = data[::slc, 2]

n = x.shape[0]
print("Points: ", n)

ds = np.sqrt(np.diff(x, prepend=0)**2 + np.diff(y, prepend=0)**2)
s = np.cumsum(ds)
dt = np.diff(t, prepend=0)

window_width = 100
window_size = int(window_width / (s[-1] / n))
whalf = int(window_size / 2)

v_max = 100
opt_speed = np.zeros(n)
value_counts = np.zeros(n)
speeds = []
plt.figure(figsize=(10, 10))
plt.pause(3)
while True:
    start = np.random.randint(-whalf, n + whalf)
    end = start + window_size
    if start < 0:
        start = 0
    elif end > n:
        end = n
    if end - start < 2:
        continue
    sw = s[start:end]
    xw = x[start:end]
    yw = y[start:end]

    v_low = 0
    v_high = v_max
    best_v = None
    threshold = 0.4
    while v_high - v_low > 1:
        v_mean = (v_low + v_high) / 2
        l = 0.92 * v_mean - 8.72
        kernel = RBF(length_scale=l, length_scale_bounds='fixed')
        noise_std = 0.01
        gp = GaussianProcessRegressor(kernel=kernel, optimizer=None, alpha=noise_std**2, normalize_y=True)

        gp.fit(sw.reshape(-1, 1), xw)
        x_pred = gp.predict(sw.reshape(-1, 1))
        x_residual = np.abs(xw - x_pred)

        gp.fit(sw.reshape(-1, 1), yw)
        y_pred = gp.predict(sw.reshape(-1, 1))
        y_residual = np.abs(yw - y_pred)

        residual = (np.sqrt(x_residual**2 + y_residual**2)*2).mean()

        if residual <= threshold:
            best_v = v_mean
            v_low = v_mean
        else:
            v_high = v_mean

    # weighted average for the optimal speed
    opt_speed[start:end] = (v_mean + opt_speed[start:end] * value_counts[start:end]) / (1 + value_counts[start:end])
    value_counts[start:end] += 1
    speeds.append(v_mean)
    
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    lc = LineCollection(segments, cmap='viridis', norm=plt.Normalize(vmin=opt_speed.min(), vmax=opt_speed.max()))
    lc.set_array(opt_speed)
    lc.set_linewidth(10)

    plt.clf()
    ax = plt.gca()
    ax.add_collection(lc)
    cbar = plt.colorbar(lc, label='Speed (m/s)', fraction=0.036, pad=0.04)
    ax.scatter(xw, yw, c='red', s=35, label='Window', zorder=5)
    ax.autoscale()
    ax.set_aspect('equal')
    plt.pause(0.1)







